# Computing climate indicators

This notebook will get you started on the use of `xclim` to subset netCDF arrays and compute climate indicators, taking advantage of parallel processing capabilities offered by `xarray` and `dask`. 

`xarray` is a python package making it easy to work with n-dimensional arrays. It labels axes with their names (time, lat, lon, level) instead of indices (0,1,2,3), reducing the likelihood of bugs and making the code easier to understand. One of the key strengths of `xarray` is that it knows how to deal with non-standard calendars (I'm looking at you 360_days) and can easily resample daily time series to weekly, monthly, seasonal or annual periods.  Finally, `xarray` is tightly inegrated with `dask`, a package that can automatically parallelize operations.


In [14]:
# XCLIM and xarray
import xclim.indices as xci
import xclim.atmos as atmos
import numpy as np
import xarray as xr
import dask

# file handling libraries
import os
import glob
import time
import tempfile
from pathlib import Path

# Output folder
outfolder = Path(tempfile.mkdtemp()) 

## 1. Setting up the Dask client - Parallel processing / workers

First we create a pool of workers that will wait for jobs. The xarray library will automatically connect to these workers and and dispatch them jobs that can be run in parallel. 

The dashboard link lets you see in real time how busy those workers are. 

In [7]:
from distributed import Client
client=Client(n_workers=2, threads_per_worker=12, dashboard_address=8787, memory_limit='6GB') 
#client=Client(n_workers=1)
client

DEBUG:asyncio:Using selector: EpollSelector
DEBUG:asyncio:Using selector: EpollSelector


Client Scheduler: tcp://127.0.0.1:33109 Dashboard: http://127.0.0.1:35955/status,Cluster Workers: 2 Cores: 24 Memory: 12.00 GB


## 2. Finding data files 

In [2]:
infolder = '/scen3/scenario/netcdf/ouranos/cb-oura-1.0/'

# Get list of files for tasmax
rcps = ['rcp45','rcp85']
v = 'tasmax'
r = rcps[0]
search_str = os.path.join(infolder, '{v}*CanESM*{r}*.nc'.format(v=v,r=r))
sim_files= sorted(glob.glob(search_str))
print(len(sim_files))

151


## 3. Creating xarray datasets

To open a netCDF file with `xarray`, we use `xr.open_dataset(<path to file>)`. But by default, the entire file is stored in one chunk, so there is no parallelism. To trigger parallel computations, we need to explicitly specify the *chunk* size. 

`Dask`' parallelism is based on memory chunks. We need to tell `xarray` to split our netCDF array into chunks of a given size, and operations on each chunk of the array will automatically be dispatched to the workers. 

In [3]:
# This file is opened as one big chunk: no parallel processing. 
ds = xr.open_dataset(sim_files[0])
print(ds.tasmax)

<xarray.DataArray 'tasmax' (time: 365, lat: 700, lon: 1064)>
[271852000 values with dtype=float32]
Coordinates:
  * time     (time) object 1950-01-01 00:00:00 ... 1950-12-31 00:00:00
  * lat      (lat) float32 83.28931 83.20598 83.12265 ... 25.12497 25.04164
  * lon      (lon) float32 -141.04314 -140.9598 ... -52.54667 -52.46334
Attributes:
    units:          K
    long_name:      air_temperature
    standard_name:  air_temperature


In [4]:
# Chunked in memory along the time dimension.
# Note that the data type is a 'dask.array'. xarray will automatically use client workers 
ds = xr.open_dataset(sim_files[0], chunks={'time': 31})
print(ds.tasmax)

<xarray.DataArray 'tasmax' (time: 365, lat: 700, lon: 1064)>
dask.array<shape=(365, 700, 1064), dtype=float32, chunksize=(31, 700, 1064)>
Coordinates:
  * time     (time) object 1950-01-01 00:00:00 ... 1950-12-31 00:00:00
  * lat      (lat) float32 83.28931 83.20598 83.12265 ... 25.12497 25.04164
  * lon      (lon) float32 -141.04314 -140.9598 ... -52.54667 -52.46334
Attributes:
    units:          K
    long_name:      air_temperature
    standard_name:  air_temperature


### 3.1. Multifile dataset
netCDF files are often split into periods to keep file size manageable. A single dataset can be split in dozens of individual files. `xarray` has a function `open_mfdataset` that can open and aggregate a list of files and construct a unique *logical* dataset. `open_mfdataset` can aggregate files over coordinates (time, lat, lon) and variables. 

Note that opening a multi-file dataset automatically chunks the array (one chunk per file).

Note also that because `xarray` reads every file metadata to place it in a logical order, it can take a while to load. 

In [11]:
sim_files

['/scen3/scenario/netcdf/ouranos/cb-oura-1.0/tasmax_day_CanESM2_rcp45_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1950.nc',
 '/scen3/scenario/netcdf/ouranos/cb-oura-1.0/tasmax_day_CanESM2_rcp45_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1951.nc',
 '/scen3/scenario/netcdf/ouranos/cb-oura-1.0/tasmax_day_CanESM2_rcp45_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1952.nc',
 '/scen3/scenario/netcdf/ouranos/cb-oura-1.0/tasmax_day_CanESM2_rcp45_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1953.nc',
 '/scen3/scenario/netcdf/ouranos/cb-oura-1.0/tasmax_day_CanESM2_rcp45_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1954.nc',
 '/scen3/scenario/netcdf/ouranos/cb-oura-1.0/tasmax_day_CanESM2_rcp45_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1955.nc',
 '/scen3/scenario/netcdf/ouranos/cb-oura-1.0/tasmax_day_CanESM2_rcp45_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1956.nc',
 '/scen3/scenario/netcdf/ouranos/cb-oura-1.0/tasmax_day_CanESM2_rcp45_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1957.nc',
 '/scen3/scenario/netcdf

In [5]:
# Create multi-file data & chunks 
ds = xr.open_mfdataset(sim_files, chunks={'time':365, 'lat':50*2, 'lon':56*2})
ds = ds.drop('time_vectors')
ds = ds.drop('ts')
print(ds)

<xarray.Dataset>
Dimensions:  (lat: 700, lon: 1064, time: 55115)
Coordinates:
  * lat      (lat) float32 83.28931 83.20598 83.12265 ... 25.12497 25.04164
  * lon      (lon) float32 -141.04314 -140.9598 ... -52.54667 -52.46334
  * time     (time) object 1950-01-01 00:00:00 ... 2100-12-31 00:00:00
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<shape=(55115, 700, 1064), chunksize=(365, 100, 112)>
Attributes:
    Conventions:     CF-1.5
    title:           CanESM2 model output prepared for CMIP5 historical
    history:         2011-04-14T00:21:01Z CMOR rewrote data to comply with CF...
    institution:     CCCma (Canadian Centre for Climate Modelling and Analysi...
    source:          CanESM2 2010 atmosphere: CanAM4 (AGCM15i, T63L35) ocean:...
    redistribution:  Redistribution prohibited. For internal use only.


## 4. Subsetting utilities

### subset_bbox : using a latitude-longitude bounding box

In [6]:
from xclim import subset
lat_bnds = [45, 60]
lon_bnds = [-55, -82]

ds1 = subset.subset_bbox(ds,lat_bnds=lat_bnds,lon_bnds=lon_bnds)
print(ds1)

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 324, time: 55115)
Coordinates:
  * lat      (lat) float64 59.96 59.87 59.79 59.71 ... 45.29 45.21 45.12 45.04
  * lon      (lon) float64 -81.96 -81.88 -81.8 -81.71 ... -55.21 -55.13 -55.05
  * time     (time) object 1950-01-01 00:00:00 ... 2100-12-31 00:00:00
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<shape=(55115, 180, 324), chunksize=(365, 20, 75)>
Attributes:
    Conventions:     CF-1.5
    title:           CanESM2 model output prepared for CMIP5 historical
    history:         2011-04-14T00:21:01Z CMOR rewrote data to comply with CF...
    institution:     CCCma (Canadian Centre for Climate Modelling and Analysi...
    source:          CanESM2 2010 atmosphere: CanAM4 (AGCM15i, T63L35) ocean:...
    redistribution:  Redistribution prohibited. For internal use only.


### Add start and/or end years

Note that in the next release, we'll use datetime objects instead of a year integer to specify start and end points.

In [7]:
ds2 = subset.subset_bbox(ds,lat_bnds=lat_bnds,lon_bnds=lon_bnds, start_yr=1981, end_yr=2010)
print(ds2)
print(' ')

# subset years only
ds2 = subset.subset_bbox(ds, start_yr=1981, end_yr=2010)
print(ds2)

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 324, time: 10950)
Coordinates:
  * lat      (lat) float64 59.96 59.87 59.79 59.71 ... 45.29 45.21 45.12 45.04
  * lon      (lon) float64 -81.96 -81.88 -81.8 -81.71 ... -55.21 -55.13 -55.05
  * time     (time) object 1981-01-01 00:00:00 ... 2010-12-31 00:00:00
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<shape=(10950, 180, 324), chunksize=(365, 20, 75)>
Attributes:
    Conventions:     CF-1.5
    title:           CanESM2 model output prepared for CMIP5 historical
    history:         2011-04-14T00:21:01Z CMOR rewrote data to comply with CF...
    institution:     CCCma (Canadian Centre for Climate Modelling and Analysi...
    source:          CanESM2 2010 atmosphere: CanAM4 (AGCM15i, T63L35) ocean:...
    redistribution:  Redistribution prohibited. For internal use only.
 
<xarray.Dataset>
Dimensions:  (lat: 700, lon: 1064, time: 10950)
Coordinates:
  * lat      (lat) float64 83.29 83.21 83.12 83.04 ... 25.29 25.21 25.12 

### Select a single grid point 

In [8]:
lon_pt = -70.0
lat_pt = 50.0

ds3 = subset.subset_gridpoint(ds,lon=lon_pt,lat=lat_pt, start_yr=1981)
print(ds3)

<xarray.Dataset>
Dimensions:  (time: 43800)
Coordinates:
    lat      float32 50.04064
    lon      float32 -69.96264
  * time     (time) object 1981-01-01 00:00:00 ... 2100-12-31 00:00:00
Data variables:
    tasmax   (time) float32 dask.array<shape=(43800,), chunksize=(365,)>
Attributes:
    Conventions:     CF-1.5
    title:           CanESM2 model output prepared for CMIP5 historical
    history:         2011-04-14T00:21:01Z CMOR rewrote data to comply with CF...
    institution:     CCCma (Canadian Centre for Climate Modelling and Analysi...
    source:          CanESM2 2010 atmosphere: CanAM4 (AGCM15i, T63L35) ocean:...
    redistribution:  Redistribution prohibited. For internal use only.


### Nothing has been computed so far !

If you look at the output of those operations, they're identified as `dask.array` objects. What happens is that `dask` creates a chain of operations that when executed, will yield the values we want. But as long as we don't explicitly ask for a value, no computation will occur. 

You can trigger computations by using the `load` or `compute` method, or writing the output to disk. 

## 5. Climate index calculation & resampling frequencies

`xclim` has two layers for the calculation of indicators. The bottom layer is composed of a list of functions that take a `xarray.DataArray` as an input and return an `xarray.DataArray` as output. You'll find these functions in `xclim.indices`. The indicator's logic is contained in this function, as well as potential unit conversions, but it doesn't check if the time frequency is daily, and doesn't not adjust the meta data of the output array. 

The second layer are class instances that you'll find organized by *realm*. So far, there is only one realm (atmospheric) available in `xclim.atmos`, but we'll be working on `ice` and `land` indicators in 2020. Before running computations, these classes check the input data is a daily average of the expected variable. If an indicator expects a daily mean and you pass it a daily max, an error will be raised. After the computation, it also checks the number of values per period to make sure there are not missing values or NaNs in the input data. If there are, the output is going to be set to NaN. The output units are set correctly as well as other properties of the output array, complying as much as possible with CF conventions. 

For new users, we suggest you use the classes found in `xclim.atmos`. If you know what you're doing and you want to circumvent the built-in checks, then you can use the `xclim.indices` directly. 

All `xclim` indicators convert daily data to lower time frequencies, such as monthly or annual values. This is done using `xarray.DataArray.resample` method. Resampling creates a grouped object over which you apply a reduction operation (e.g. mean, min, max). The list of available frequency is given in the link below, but the most often used are: 

- YS: annual starting in January
- YS-JUL: annual starting in July
- MS: monthly
- QS-DEC: seasonal starting in December
- W: weekly
- 2W: bi-weekly

http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases


In the example below, we're computing the annual maximum temperature of the daily maximum temperature.

In [13]:
fr = 'YS'
out = atmos.tx_max(ds1.tasmax, freq=fr)

# Why are you doing this? This is confusing I think. 
dsOut = ds1.drop(ds1.data_vars)
dsOut = dsOut.drop('time')
dsOut[out.name] = out
print('Number of time-steps using freq == ', fr, ' : ', len(out.time),'\n')
print(out.time)
print(dsOut)

# ?
outfile = '/home/logan/boreas/testdata/XCLIM_workshop/Exercise1/rcp45/tx_max/test.nc'
if not os.path.exists(os.path.dirname(outfile)):
    os.makedirs(os.path.dirname(outfile))
print(outfile)


  warn(e)



Number of time-steps using freq ==  YS  :  151 

<xarray.DataArray 'time' (time: 151)>
array([cftime.DatetimeNoLeap(1950, 1, 1, 0, 0, 0, 0, 4, 1),
       cftime.DatetimeNoLeap(1951, 1, 1, 0, 0, 0, 0, 5, 1),
       cftime.DatetimeNoLeap(1952, 1, 1, 0, 0, 0, 0, 6, 1),
       cftime.DatetimeNoLeap(1953, 1, 1, 0, 0, 0, 0, 0, 1),
       cftime.DatetimeNoLeap(1954, 1, 1, 0, 0, 0, 0, 1, 1),
       cftime.DatetimeNoLeap(1955, 1, 1, 0, 0, 0, 0, 2, 1),
       cftime.DatetimeNoLeap(1956, 1, 1, 0, 0, 0, 0, 3, 1),
       cftime.DatetimeNoLeap(1957, 1, 1, 0, 0, 0, 0, 4, 1),
       cftime.DatetimeNoLeap(1958, 1, 1, 0, 0, 0, 0, 5, 1),
       cftime.DatetimeNoLeap(1959, 1, 1, 0, 0, 0, 0, 6, 1),
       cftime.DatetimeNoLeap(1960, 1, 1, 0, 0, 0, 0, 0, 1),
       cftime.DatetimeNoLeap(1961, 1, 1, 0, 0, 0, 0, 1, 1),
       cftime.DatetimeNoLeap(1962, 1, 1, 0, 0, 0, 0, 2, 1),
       cftime.DatetimeNoLeap(1963, 1, 1, 0, 0, 0, 0, 3, 1),
       cftime.DatetimeNoLeap(1964, 1, 1, 0, 0, 0, 0, 4, 1),
       cftime

PermissionError: [Errno 13] Permission denied: '/home/logan'

## 7. xclim computations are *lazy*

Up until now we have ony created a schedule of tasks with a small preview, not done any actual computations. As mentionned above, writing the output to disk will trigger the cascade of computations on all the chunks. 

In [15]:
outfile = outfolder / 'test_tx_max.nc'
start= time.time()
dsOut.to_netcdf(outfile, format='NETCDF4')
end = time.time()
print('calculation took ',end-start, 's')

  return func(*args, **kwargs)

  ret = f(ret)



calculation took  298.7508809566498 s


### Optimizing the chunk size

You can improve performance by being smart about chunk sizes. If chunks are too small, there is a lot of time lost in overhead. If chunks are too large, you may end up with less workers. 

In [16]:
print(ds1)


<xarray.Dataset>
Dimensions:  (lat: 180, lon: 324, time: 55115)
Coordinates:
  * lat      (lat) float64 59.96 59.87 59.79 59.71 ... 45.29 45.21 45.12 45.04
  * lon      (lon) float64 -81.96 -81.88 -81.8 -81.71 ... -55.21 -55.13 -55.05
  * time     (time) object 1950-01-01 00:00:00 ... 2100-12-31 00:00:00
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<shape=(55115, 180, 324), chunksize=(365, 20, 75)>
Attributes:
    Conventions:     CF-1.5
    title:           CanESM2 model output prepared for CMIP5 historical
    history:         2011-04-14T00:21:01Z CMOR rewrote data to comply with CF...
    institution:     CCCma (Canadian Centre for Climate Modelling and Analysi...
    source:          CanESM2 2010 atmosphere: CanAM4 (AGCM15i, T63L35) ocean:...
    redistribution:  Redistribution prohibited. For internal use only.


In [17]:
ds1 = ds1.chunk(chunks={'time':365, 'lon':-1, 'lat':-1})
print(ds1)

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 324, time: 55115)
Coordinates:
  * lat      (lat) float64 59.96 59.87 59.79 59.71 ... 45.29 45.21 45.12 45.04
  * lon      (lon) float64 -81.96 -81.88 -81.8 -81.71 ... -55.21 -55.13 -55.05
  * time     (time) object 1950-01-01 00:00:00 ... 2100-12-31 00:00:00
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<shape=(55115, 180, 324), chunksize=(365, 180, 324)>
Attributes:
    Conventions:     CF-1.5
    title:           CanESM2 model output prepared for CMIP5 historical
    history:         2011-04-14T00:21:01Z CMOR rewrote data to comply with CF...
    institution:     CCCma (Canadian Centre for Climate Modelling and Analysi...
    source:          CanESM2 2010 atmosphere: CanAM4 (AGCM15i, T63L35) ocean:...
    redistribution:  Redistribution prohibited. For internal use only.


In [18]:
out = atmos.tx_max(ds1.tasmax, freq=fr)
dsOut = ds1.drop(ds1.data_vars)
dsOut = dsOut.drop('time')
dsOut[out.name] = out
outfile = os.path.join(outfolder, 'test.nc')
start= time.time()

dsOut.to_netcdf( outfile,format='NETCDF4')

end = time.time()
print('calculation took ',end-start, 's')

  warn(e)

  return func(*args, **kwargs)

  ret = f(ret)



calculation took  165.12852549552917 s
